In [7]:
using Distributions

## Parameters to get S
Get the parameters for the N that is our frequency random varible and we know it distributes as N $\sim$ NegativeBinomial(r, p) but we also know that $\mathbb{E} = 140$ and also that $\mathbb{V} = 15²$.

Doing a little reasherch we can get that the Expected value of N is $\frac{r(1-p)}{p}$ and its variance is $\mathbb{V}(N) = \frac{r(1-p)}{p²}$
So it's only left to plug-in our values and solve for $r$ and $p$. That gives us that $r = 3920/17$ and $p = 28/45$ and we don't need to round the value of r beacuse in julia it's an expression for the distribution in case $r \in \mathbb{R}$

For Y we know that it is distributed as a $LogNormal(\mu, \sigma)$ and also that $Median(Y) = 50$ and it's $VaR_{0.995}(Y) = 5500$ 
Usign a web graphical calculator we ge to the aproximate value of: $\mu = 3.912115$ $\sigma = 2.71873$ but we can get a better approximation if we use the EDA Algorith 

In [5]:
include("EDA.jl")

EDA

In [70]:
obj_func(x) = (cdf(LogNormal(x[1], x[2]), 50) - 0.5)^2 + (cdf(LogNormal(x[1], x[2]), 5500) - 0.995)^2
eda_res = EDA(obj_func,[2,2],[4,3], difmax = 0.000000000001)

Iterando... 
...fin
Aviso: se alcanzó el máximo número de iteraciones = 1000


(x = [3.911206439323575, 1.9526323965802121], fx = 9.19246792476585e-6, iter = 1000)

In [73]:
println(cdf(LogNormal(eda_res[1][1], eda_res[1][2]), 5500))
println(cdf(LogNormal(eda_res[1][1], eda_res[1][2]), 50))

0.9919726852147105
0.5001668325973346



Now we have the parameters for both of the distributions and we can go on

In [75]:
mu = eda_res[1][1]; sigma = eda_res[1][2]; 

freq = NegativeBinomial(3920/17 ,28/45)
sev = LogNormal(mu, sigma)

LogNormal{Float64}(μ=3.911206439323575, σ=1.9526323965802121)

## Simulations of the accidents
Now thtat we have the parameters of the severity and the frequency distributions we now can simulate de variable S

In [79]:
function  simul_events(n)
    S = zeros(n)
    N = rand(freq, n)
    
    N_pos = findall(N .> 0)
    
    for i in N_pos
        Yi = rand(sev, N[i])
        S[i] = sum(Yi)
    end
    return(S, N)
end

simul_events (generic function with 1 method)

In [80]:
#  The exercise ask for 1M simulations
S = simul_events(10_000)

([51900.97038709671, 170096.0126631323, 33991.36602915016, 46092.520160729546, 37765.10992240988, 33611.78354764529, 50498.7920713216, 75121.41579461901, 16331.500108666842, 25915.449892703822  …  35647.44355539508, 39008.007093661116, 46712.269615274156, 85494.78619406313, 39334.18725587172, 42951.22329305191, 91168.87638778181, 35310.76053641446, 44797.10892185627, 46136.095820204144], [156, 133, 148, 136, 144, 138, 130, 156, 113, 132  …  120, 148, 144, 141, 114, 142, 142, 143, 140, 142])

The next step is to fit the distribution of S into a prior T-Student distribution as $Student(\mu_{0}, n_{0}\alpha(n_{0} + 1)^{-1}\beta^{-1}, 2\alpha)$

We viewed the for of a 3-parameter T-student distribution in class so ownow we will use it and aproximate the parameters.
remember that:

If $X \sim ST(\alpha)$ entonces $f_{X}(x | \alpha) = \frac{\Gamma(\frac{\alpha + 1}{2})}{\sqrt{\alpha\pi}\ \Gamma(\frac{\alpha}{2})} * (1 + \frac{x^{2}}{\alpha})^{\frac{\alpha +1}{2}}$ 

and usign a location-scale parametrization:

Let $\mu \in \mathbb{R}$, $\lambda > 0$ and  $T := \mu + \frac{1}{\lambda} X$
with
$\mathbb{E}[T] = \mu$,  $Median[T] = \mu$  and just in the case that  $\alpha > 2$  then   $\mathbb{V}[T] = \frac{\alpha}{(\alpha - 2)\lambda}$

So our new dsitribution function:
$f_{T}(t| \mu, \lambda, \alpha) = \sqrt{\lambda}\ f_{X}(\sqrt{\lambda} (t -\mu)\ | \alpha)$

In our case we can use these parametres as:

$\mathbb{E}(S) = \mu_{0}$ and the $\mathbb{E}(S) = n_{0}$

So our values to adjust are:

$\mu = \mu_{0}$ 

$\lambda = n_{0}\alpha(n_{0} + 1)^{-1}\beta^{-1}$ 

$\alpha = 2\alpha$ 

In [81]:
μ0 = mean(S[1])
n0 = mean(S[2])

139.823